# 1. 準備

### 1.1. 基本設定

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.dates as mdates
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler


# カレントディレクトリを.pyと合わせるために以下を実行
if Path.cwd().name == "notebook":
    os.chdir("..")

# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)

# 浮動小数点数を小数点以下3桁で表示するように設定
pd.set_option('display.float_format', '{:.3f}'.format)

In [ ]:
# Mac Matplotlibのデフォルトフォントをヒラギノ角ゴシックに設定
plt.rcParams['font.family'] = 'Hiragino Sans'

In [ ]:
# Windows MatplotlibのデフォルトフォントをMeiryoに設定
plt.rcParams['font.family'] = 'Meiryo'

### 1.2. csv読み込み

In [ ]:
# point_history.csvの読み込み
df_point_history_sorce = pd.read_csv('data/input/point_history_cleansing.csv')

In [ ]:
# ユーザー基本情報の読み込み
df_user_base_sorce = pd.read_csv("data/input/user_info_merged.csv")

### 1.3. データクレンジング

#### 1.3.1. df_user_base(ユーザ基本情報)のクレンジング

In [ ]:
# DataFrameのコピーを作成
feature_list = [
    'id',
    'club_coin',
    'recycle_point',
    'total_recycle_amount',
    'recycle_amount_per_year',
    'recycle_amount_after_gold_member',
    'rank_continuation_class',
    'gender',
    '緯度',
    '経度',
    '登録日時',
    'カード登録',
    '最終利用日',
    '登録店舗との距離',
    '毎月平均リサイクル量',
    '毎月平均リサイクル回数',
    '毎月平均クラブコインの使用量',
    '毎月平均ガチャの取得量',
    '毎月平均ガチャの使用量',
    '平均rank',
    'サービス利用開始からの経過日数',
    'birthday'
    ]

df_user_base = df_user_base_sorce.copy()
df_user_base = df_user_base[feature_list]

# 紛らわしい列名を改名
df_user_base = df_user_base.rename(columns={'登録日時': 'アプリ登録日時', '最終利用日': 'アプリ最終利用日'})

# objectをdatetimeに変更
df_user_base['アプリ登録日時'] = pd.to_datetime(df_user_base['アプリ登録日時'], errors='coerce')
df_user_base['アプリ最終利用日'] = pd.to_datetime(df_user_base['アプリ最終利用日'], errors='coerce')
df_user_base['カード登録'] = pd.to_datetime(df_user_base['カード登録'], errors='coerce')
df_user_base['アプリ最終利用日'] = pd.to_datetime(df_user_base['アプリ最終利用日'], errors='coerce')
df_user_base['birthday'] = pd.to_datetime(df_user_base['birthday'], errors='coerce')

# 6歳未満(1543個)と100歳以上(12個)を削除
df_user_base = df_user_base[ (df_user_base['birthday'] < pd.to_datetime('2017-01-01')) & (df_user_base['birthday'] > pd.to_datetime('1924-01-01'))]

# df_user_baseに"age"と"age_group"のカラムを追加
df_user_base['age'] = pd.Timestamp.now().year - df_user_base['birthday'].dt.year    # ageの算出・追加

df_user_base = df_user_base.sort_values(by='アプリ登録日時')

郵便局上3桁でワンホットエンコーディング

#### 1.3.2. df_point_history(point_history.csv)のクレンジング

## TODO:　store_latitude,store_longitudeと、userの緯度経度から、利用店舗との距離を算出してカラムに追加する

In [ ]:
# DataFrameのコピーを作成
df_point_history = df_point_history_sorce.copy()

# objectをdatetimeに変更
df_point_history['use_date'] = pd.to_datetime(df_point_history['use_date'], errors='coerce')

feature_list_point = [
    'user_id',
    'super',
    'status',
    'shop_name_1',
    'amount_kg',
    'rank_id',
    'use_date',
    'store_latitude',
    'store_longitude',
    ]
df_point_history = df_point_history[feature_list_point]
df_point_history = df_point_history.sort_values(by='use_date')

# statusが1以外は削除
df_point_history = df_point_history[df_point_history['status'] == 1]

# amount_kgが0未満は削除
df_point_history = df_point_history[df_point_history['amount_kg'] > 0]

#### 1.3.3. 分析に必要なカラムの作成

継続利用期間（point_historyのuse_date列からRPS最終利用日を抽出したver.）　231228 norosen

In [ ]:
# 各利用者id に対して「RPS利用開始日」「RPS最終利用日」を抽出
first_entries_RPS = df_point_history.groupby('user_id').first().reset_index()
last_entries_RPS = df_point_history.groupby('user_id').last().reset_index()

In [ ]:
# df_user_baseに利用開始日をマージ
df_user_base = pd.merge(df_user_base, first_entries_RPS[['user_id', 'use_date']], left_on='id', right_on='user_id', how='left')
df_user_base = df_user_base.rename(columns={'use_date':'RPS利用開始日'})

# df_user_baseに最終利用日をマージ
df_user_base = pd.merge(df_user_base, last_entries_RPS[['user_id', 'use_date']], left_on='id', right_on='user_id', how='left')
df_user_base = df_user_base.rename(columns={'use_date':'RPS最終利用日'})


df_user_base['RPS利用開始日'] = pd.to_datetime(df_user_base['RPS利用開始日'], errors='coerce')
df_user_base['RPS最終利用日'] = pd.to_datetime(df_user_base['RPS最終利用日'], errors='coerce')

In [ ]:
df_user_base = df_user_base.drop(columns=['user_id_x', 'user_id_y'])

In [ ]:
# RPS継続利用期間を計算
df_user_base['RPS継続利用期間(月)'] = (df_user_base['RPS最終利用日'] - df_user_base['RPS利用開始日']).dt.days / 30  # 月単位で計算
df_user_base = df_user_base[df_user_base['RPS継続利用期間(月)'] >= 0]

In [ ]:
# 2022年11月1日以前のデータをフィルタリング
df_user_base = df_user_base[df_user_base['RPS利用開始日'] <= pd.Timestamp('2022-11-01')]

#### 1.3.4. マージ

In [ ]:
# 月ごとにamount_kgを合計して、カラムに追加
monthly_grouped_point = df_point_history.groupby(['user_id', df_point_history['use_date'].dt.to_period('M')])['amount_kg'].sum()
monthly_grouped_point = monthly_grouped_point.reset_index()

In [ ]:
pivot_df = monthly_grouped_point.pivot(index='user_id', columns='use_date', values='amount_kg')

In [ ]:
merged_df = pd.merge(pivot_df, df_user_base,  left_on='user_id', right_on='id', how='inner')

In [ ]:
# merged_df['use_date'] = pd.to_datetime(merged_df['use_date'])
# merged_df['use_year'] = merged_df['use_date'].dt.year
# merged_df['use_month'] = merged_df['use_date'].dt.month
merged_df = merged_df.drop(columns = ['birthday',
                                      'RPS利用開始日',
                                      'RPS最終利用日',
                                      'アプリ最終利用日',
                                      'アプリ登録日時',
                                      'カード登録',
                                     ])

In [ ]:
merged_df = pd.get_dummies(merged_df,columns=['gender'])

In [ ]:
merged_df = merged_df.astype(float)

#### 1.3.5. マージ後のカラム作成

recycle amount per year  
　→　マージした後、2023年10月～2022年10月までの期間の平均に変更  
　　　　（ゆくゆくは、新規ユーザに適用するなら、3か月前など期間を狭める  


#### 標準化

In [ ]:
merged_df.columns = merged_df.columns.astype(str)

In [ ]:
# merged_df = merged_df[['平均rank','RPS継続利用期間(月)','毎月平均ガチャの取得量',
#                         '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
#                          '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
#                           '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
#                            '2023-07', '2023-08', '2023-09', '2023-10', '2023-11']]

In [ ]:
merged_df = merged_df.drop(columns = ['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10',
                                      '2020-11', '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08',
                                      '2021-09', '2021-10', '2021-11', '2021-12', 
                                      '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10',
                                      '2022-11', '2022-12', '2023-12',
                                      'gender_女', 'gender_無回答', 'gender_男', '毎月平均ガチャの使用量', '毎月平均ガチャの取得量',
                                      'recycle_amount_after_gold_member', 'id', 'club_coin', 'recycle_point', '登録店舗との距離', '毎月平均クラブコインの使用量'
                                      
                                     ])

In [ ]:
merged_df = merged_df.fillna(0)

In [ ]:
# 2023-11カラムの値が0以下または50以上の行を削除
# merged_df = merged_df[~merged_df.apply(lambda x: (x <= 0).any() or (x >= 50).any(), axis=1)]
# merged_df = merged_df[(merged_df['2023-11'] > 0) & (merged_df['2023-11'] < 50)]
# merged_df = merged_df[(merged_df['2023-11'] > 0)]
# merged_df = merged_df[~merged_df.apply(lambda x: (x >= 50).any(), axis=1)]

# 値が50以上の行を削除
columns_to_check = ['2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
                    '2023-07', '2023-08', '2023-09', '2023-10', '2023-11'
                   ]

merged_df = merged_df[~merged_df[columns_to_check].gt(20).any(axis=1)]

In [ ]:
# 'for'文を使わずに拡張されたデータフレームを作成
fill_mean_df = pd.DataFrame(merged_df)
# 平均値の計算に使用する日付カラムのみを選択
date_columns_no_loop = fill_mean_df.columns[:11]
# 日付カラム内の0をNaNに置換
date_df_no_loop = fill_mean_df[date_columns_no_loop].replace(0, np.nan)
# NaNを除外して日付カラムの各行ごとの平均値を計算
row_mean_no_loop = date_df_no_loop.mean(axis=1)

for col in date_columns_no_loop:
    fill_mean_df[col] = date_df_no_loop[col].fillna(row_mean_no_loop)

fill_mean_df = fill_mean_df.dropna(how = 'any')

In [ ]:
# StandardScalerのインスタンスを作成
scaler = StandardScaler()

# データフレームの全列を標準化
# ここでは、ID列など、標準化不要な列は除外する必要があります
columns_to_scale = merged_df.columns.difference(['gender_女',
                                               'gender_無回答',
                                               'gender_男'])
fill_mean_df[columns_to_scale] = scaler.fit_transform(fill_mean_df[columns_to_scale])

In [ ]:
fill_mean_df.info()

#### Xとyに分割

In [ ]:
X = fill_mean_df.drop(['2023-11'], axis=1)
y = fill_mean_df['2023-11']

# 2. 予測

#### trainとtestに分割

In [ ]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

evals_result = {}  # 学習過程を記録するための辞書

# LightGBMのパラメータ設定
lgb_params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'seed': 0,
    'num_iterations' : 1000, # 値を小さくするとよい
    'learning_rate' : 0.02,
    'max_depth': 10,
    'num_leaves': 10,
    'metric': 'rmse'
}

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

#### 学習

In [ ]:
# トレーニングの進捗を表示するコールバックを追加
model = lgb.train(
    lgb_params, 
    train_data, 
    valid_sets=[train_data, test_data], 
    callbacks=[
        lgb.callback.record_evaluation(evals_result)
    ]
)

In [ ]:
# RMSE の学習曲線をプロット
plt.plot(evals_result['training']['rmse'], label='train')
plt.plot(evals_result['valid_1']['rmse'], label='val')
plt.ylabel('RMSE')
plt.xlabel('Boosting round')
plt.title('Training and Validation RMSE')
plt.legend()
plt.show()

#### 予測

In [ ]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

#### 評価

In [ ]:
# Evaluate the predictions
# Calculate and print evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R2): {r2}')

print("actual")
print(y_test[:10].values)
print("pred")
print(y_pred[:10])

In [ ]:
X_train.head()

In [ ]:
scale = scaler.scale_
mean = scaler.mean_

In [ ]:
#スケールをもとに戻す
y_pred_rescale  = y_pred*scaler.scale_[10] + scaler.mean_[10]
y_train_rescale = y_train*scaler.scale_[10] + scaler.mean_[10]
y_test_rescale  = y_test*scaler.scale_[10] + scaler.mean_[10]

In [ ]:
scale = np.delete(scale, 10)
mean = np.delete(mean, 10)

In [ ]:
X_train_rescale = X_train*scale + mean
X_test_rescale = X_test*scale + mean

In [ ]:
# numpy.ndarray を pandas.Series に変換
y_test_rescale_series = pd.Series(y_test_rescale, name='Real_Recycle_Amount')
y_pred_rescale_series = pd.Series(y_pred_rescale, name='Predicted_Recycle_Amount')

# 結合（インデックスを無視）
combined_df = pd.concat([X_test_rescale.reset_index(), y_test_rescale_series.reset_index(), y_pred_rescale_series.reset_index()], axis=1, ignore_index=True)

In [ ]:
combined_df = combined_df.drop(columns = [0, 22, 24, 11,12,13,14,15,16,17,18,19,20,21])


In [ ]:
# '23' と '25' の値が 1 以上離れている行を抽出
filtered_df = combined_df[abs(combined_df[23] - combined_df[25]) >= 5]
filtered_df

In [ ]:
# actual vs. pred の散布図
plt.figure(figsize=(5, 5))
sns.scatterplot(x=y_test_rescale, y=y_pred_rescale, alpha=0.4)
plt.title('決定係数: {}'.format(round(r2, 2)))
plt.xlabel('amount_kg_2023-11 (正解値）')
plt.ylabel('amount_kg_2023-11（予測値）')

# # 拡大用
# plt.xlim([-1,15])
# plt.ylim([-1,15])

# Plot a line representing perfect predictions
plt.plot([y_test_rescale.min(), y_test_rescale.max()], [y_test_rescale.min(), y_test_rescale.max()], color='red', lw=2, linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
# 特徴量の重要度を取得
feature_importances = model.feature_importance(importance_type='gain')

# 特徴量の重要度をプロット
plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importances, y=model.feature_name())
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()